In [1]:
from langchain.document_loaders import CSVLoader
local_doc_path = 'accident.csv'
loader = CSVLoader(file_path=local_doc_path,encoding='utf-8')
docs = loader.load()

In [2]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings() 
query="A car rear ended another car."
embed = embeddings.embed_query(query)

/home/jyx/miniconda3/envs/learnlangchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [3]:
from langchain.vectorstores import DocArrayInMemorySearch
db = DocArrayInMemorySearch.from_documents(docs, embeddings)
relative_docs = db.similarity_search(query)

In [4]:
from langchain.llms import OpenAI
rag_bot = OpenAI(temperature=0.8, top_p=0.8,max_tokens=512,frequency_penalty=1.1,presence_penalty=1.1,model='gpt-3.5-turbo-instruct')
context = "".join([doc.page_content for doc in relative_docs])
final_input = f"Context: {context}\n\nQuestion: {query}\nAnswer:"

/home/jyx/miniconda3/envs/learnlangchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [5]:
from IPython.display import Markdown
import json
json_file = 'accident.json'
try:
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
except (FileNotFoundError, json.JSONDecodeError):
    data = []
for i in range(1):
    response = rag_bot.invoke(final_input)
    Markdown(response)
    new_data = {"query": query, "response": response}
    data.append(new_data)
    with open(json_file, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)